# Análisis Exploratorio Inicial

### <font color='5E5D5D'> Descripción </font>

<i><font color='C9614B'> El objetivo de esta última notebook es crear el set de datos con el formato necesario para aplicar NFQ tal cual se estipula en el paper de referencia. No se utilizarán los usuarios que no tengan al menos 20 interacciones dentro de la web.

Al igual que se indica en el paper, únicamente se realizarán nuevas variables simples que describen el estado, tales como tiempo desde la última visita, la suma de los clics previamente realizados, la cantidad de visitas totales al sitio o el tiempo transcurrido desde el ultimo clic realizado. 


Los datos históricos corresponden a un conjunto de trayectorias, una por usuario.</font></i><br>

***

# Creación de DS final

#### Librerías

In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime
import torch

import pickle

#### Archivo

In [2]:
link = 'C:/Users/bmottini/Desktop/Master 2020/Tesis/Dataset/'
archivo = 'ds_full.csv'

In [3]:
ds_total = pd.read_csv(link+archivo,sep=',')
ds_total.head()

,compra,fecha_visita,page_serial,referrer_hash,ID_cupon_visto,ID_usuario,ID_sesion,ID_compra,fecha_reg_usuario,sexo,...,large_area_name_cupon,ken_name_cupon,small_area_name_cupon,ID_cupon_compra,cantidad_cupon_compra,fecha_compra,small_area_name_compra,latitud_usuario,longitud_usuario,Cap_tex_traducido_cupon
0,0,2012-03-28 14:15:00,7,7d3892e54acb559ae36c459978489330,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,...,関東,東京都,新宿・高田馬場・中野・吉祥寺,34c48f84026e08355dc3bd19b427f09a,NaN,NaN,NaN,NaN,NaN,Delivery service
1,0,2012-03-28 14:17:28,9,7d3892e54acb559ae36c459978489330,34c48f84026e08355dc3bd19b427f09a,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,...,関東,東京都,新宿・高田馬場・中野・吉祥寺,34c48f84026e08355dc3bd19b427f09a,NaN,NaN,NaN,NaN,NaN,Delivery service
2,0,2012-03-28 14:20:05,16,7d3892e54acb559ae36c459978489330,17c450c3b470c045d35ec22b02daa690,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,...,関西,兵庫県,兵庫,17c450c3b470c045d35ec22b02daa690,NaN,NaN,NaN,NaN,NaN,Delivery service
3,0,2012-03-28 14:23:16,18,7d3892e54acb559ae36c459978489330,91a15e6a95d09e5e01b50747833b317d,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,...,関東,埼玉県,埼玉,91a15e6a95d09e5e01b50747833b317d,NaN,NaN,NaN,NaN,NaN,Delivery service
4,0,2012-03-28 14:26:25,20,7d3892e54acb559ae36c459978489330,96fcbc8f6e45d5a2de1661eb140c6e82,d9dca3cb44bab12ba313eaa681f663eb,673af822615593249e7c6a9a1a6bbb1a,NaN,2012-03-28 14:14:18,f,...,関東,東京都,銀座・新橋・東京・上野,96fcbc8f6e45d5a2de1661eb140c6e82,NaN,NaN,NaN,NaN,NaN,Delivery service


In [4]:
# Información general archivo
ds_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2833180 entries, 0 to 2833179
Data columns (total 43 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   compra                   int64  
 1   fecha_visita             object 
 2   page_serial              int64  
 3   referrer_hash            object 
 4   ID_cupon_visto           object 
 5   ID_usuario               object 
 6   ID_sesion                object 
 7   ID_compra                object 
 8   fecha_reg_usuario        object 
 9   sexo                     object 
 10  edad                     int64  
 11  fecha_baja_usuario       object 
 12  pref_name_usuario        object 
 13  capsule_text_cupon       object 
 14  genre_name_cupon         object 
 15  descuento_cupon          float64
 16  precio_full              float64
 17  precio_dto               float64
 18  fecha_inicio_disponible  object 
 19  fecha_fin_disponible     object 
 20  dias_cupon_disponible    float64
 21  fecha_in

El paper indica que se eliminaron los usuarios que cuentan con menos de 20 interacciones. Esto puede ser interpretado como usuarios con más de 20 sesiones diferentes o como usuarios que tienen más de 20 cupones vistos.

según se indica, de los 23.000 usuarios se quedaron con 13.000 luego de esta depuración. Veamos que valores obtenemos realizando el filtro con ambas definiciones para ver si llegamos a esos valores con alguna de las dos.

In [5]:
# Determinamos la cantidad de sesiones por usuario
sesiones_usuarios = ds_total.groupby('ID_usuario')['ID_sesion'].nunique().to_frame('sesiones').reset_index()

# Eliminamos usuarios con menos de 20
sesiones_20 = sesiones_usuarios[sesiones_usuarios['sesiones'] >= 20]
print('cantidad de usuarios con 20 sesiones o mas:',len(sesiones_20))
sesiones_20.head(3)

cantidad de usuarios con 20 sesiones o mas: 9701


,ID_usuario,sesiones
3,000cc06982785a19e2a2fdb40b1c9d59,138
5,001acdee812a18acfd7509172bed5700,33
9,002822059a01d895fad84f2f2ff5c1f1,27


In [6]:
# Determinamos la cantidad de cupones vistos por usuario
cupones_usuarios = ds_total.groupby('ID_usuario')['ID_cupon_visto'].count().to_frame('cupones').reset_index()

# Eliminamos usuarios con menos de 20
cupones_20 = cupones_usuarios[cupones_usuarios['cupones'] >= 20]
print('cantidad de usuarios con 20 cupones vistos o mas:', len(cupones_20))
cupones_20.head()

cantidad de usuarios con 20 cupones vistos o mas: 13716


,ID_usuario,cupones
0,0000b53e182165208887ba65c079fc21,75
3,000cc06982785a19e2a2fdb40b1c9d59,270
4,0013518e41c416cd6a181d277dd8ca0b,29
5,001acdee812a18acfd7509172bed5700,126
6,001fd7876e3aa29393537c6baf308e43,24


Si entendemos interacción como sesiones y realizamos el filtro, obtenemos únicamente 9.701 usuarios, mientras que, si tomamos como interacción un cupón visto, obtenemos 13.716 usuarios. 

#### interacción = cupón visto

In [7]:
# creamos el Dataset con los usuraios con 20 o mas cupones vistos
ds_mas_20 = ds_total.merge(cupones_20, left_on='ID_usuario', right_on='ID_usuario', how='inner')
print('Cantidad de usuarios unicos luego del cruce:', ds_mas_20.ID_usuario.nunique())

Cantidad de usuarios unicos luego del cruce: 13716


Por otor lado, el paper hace referencia a que también se eliminan los usuarios que no registraron clics en ninguna de sus interacciones dentro del sitio. Este punto es bien importante, ya que marca fuertemente las características de los datos con los cuales el agente y su red se van a entrenar.

Para esto generaremos un set de datos en donde se hayan eliminado los usuarios con menos de 20 interacciones y que no hayan realizado ninguna compra.

In [8]:
# Determinamos la cantidad de cupones comprados por usuario
compras_usuarios = ds_mas_20.groupby('ID_usuario')['compra'].sum().to_frame('compras').reset_index()

# Nos quedamos con los usuarios con compras
usuarios_con_compra = compras_usuarios[compras_usuarios['compras'] >= 1]
print('cantidad de usuarios con al menos una compra:', len(usuarios_con_compra))
usuarios_con_compra.head()

cantidad de usuarios con al menos una compra: 12692


,ID_usuario,compras
0,0000b53e182165208887ba65c079fc21,1
1,000cc06982785a19e2a2fdb40b1c9d59,20
2,0013518e41c416cd6a181d277dd8ca0b,4
3,001acdee812a18acfd7509172bed5700,12
4,001fd7876e3aa29393537c6baf308e43,1


Obtendremos el Dataset en donde se eliminaron los usuarios con menos de 20 cupones vistos, y donde se eliminaron los usuarios que no realizaron ninguna compra

In [9]:
# creamos el dataset con los usuraios con 20 o mas cupones vistos y que al menos tienen 1 compra realizada
ds = ds_mas_20.merge(usuarios_con_compra, left_on='ID_usuario', right_on='ID_usuario', how='inner')
print('Cantidad de usuarios unicos luego del cruce:', ds.ID_usuario.nunique())

Cantidad de usuarios unicos luego del cruce: 12692


In [10]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666735 entries, 0 to 2666734
Data columns (total 45 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   compra                   int64  
 1   fecha_visita             object 
 2   page_serial              int64  
 3   referrer_hash            object 
 4   ID_cupon_visto           object 
 5   ID_usuario               object 
 6   ID_sesion                object 
 7   ID_compra                object 
 8   fecha_reg_usuario        object 
 9   sexo                     object 
 10  edad                     int64  
 11  fecha_baja_usuario       object 
 12  pref_name_usuario        object 
 13  capsule_text_cupon       object 
 14  genre_name_cupon         object 
 15  descuento_cupon          float64
 16  precio_full              float64
 17  precio_dto               float64
 18  fecha_inicio_disponible  object 
 19  fecha_fin_disponible     object 
 20  dias_cupon_disponible    float64
 21  fecha_in

In [11]:
ds.head()

,compra,fecha_visita,page_serial,referrer_hash,ID_cupon_visto,ID_usuario,ID_sesion,ID_compra,fecha_reg_usuario,sexo,...,small_area_name_cupon,ID_cupon_compra,cantidad_cupon_compra,fecha_compra,small_area_name_compra,latitud_usuario,longitud_usuario,Cap_tex_traducido_cupon,cupones,compras
0,0,2011-07-15 19:30:54,1,7d3892e54acb559ae36c459978489330,452579446692ff677b8556406133f0f4,560574a339f1b25e57b0221e486907ed,dbe81e14e993ebe6c3135b5782931ce0,NaN,2011-05-18 00:41:48,f,...,NaN,NaN,NaN,NaN,NaN,35.689521,139.691704,NaN,123,17
1,0,2011-12-13 23:20:51,2,1280d2e94c507864438af604104545e4,272698255c86c9b3c78b1abc93209476,560574a339f1b25e57b0221e486907ed,bddba586bb0116e0bde91a9678a7dc7b,NaN,2011-05-18 00:41:48,f,...,銀座・新橋・東京・上野,272698255c86c9b3c78b1abc93209476,NaN,NaN,NaN,35.689521,139.691704,Food,123,17
2,0,2011-12-22 19:10:32,2,1280d2e94c507864438af604104545e4,fe193eefc7b5a2c85f9661ce724cf266,560574a339f1b25e57b0221e486907ed,05d7c582dd5fe846a872ed6cec3d8cb8,NaN,2011-05-18 00:41:48,f,...,恵比寿・目黒・品川,fe193eefc7b5a2c85f9661ce724cf266,NaN,NaN,NaN,35.689521,139.691704,Food,123,17
3,0,2012-01-15 20:38:26,2,7d3892e54acb559ae36c459978489330,915667eb421bb294ce0c3b318512eb17,560574a339f1b25e57b0221e486907ed,5667017ad3ef436680e9b2132de464f1,NaN,2011-05-18 00:41:48,f,...,銀座・新橋・東京・上野,915667eb421bb294ce0c3b318512eb17,NaN,NaN,NaN,35.689521,139.691704,Delivery service,123,17
4,0,2012-01-15 20:39:16,4,7d3892e54acb559ae36c459978489330,87c7ca4bda4fe8465943ad50c7011982,560574a339f1b25e57b0221e486907ed,5667017ad3ef436680e9b2132de464f1,NaN,2011-05-18 00:41:48,f,...,赤坂・六本木・麻布,87c7ca4bda4fe8465943ad50c7011982,NaN,NaN,NaN,35.689521,139.691704,Food,123,17


## Ingeniería de atributos

### Modificacion variables actuales

In [12]:
# Veamos las variables que tenemos actualmente
ds.columns

Index(['compra', 'fecha_visita', 'page_serial', 'referrer_hash',
       'ID_cupon_visto', 'ID_usuario', 'ID_sesion', 'ID_compra',
       'fecha_reg_usuario', 'sexo', 'edad', 'fecha_baja_usuario',
       'pref_name_usuario', 'capsule_text_cupon', 'genre_name_cupon',
       'descuento_cupon', 'precio_full', 'precio_dto',
       'fecha_inicio_disponible', 'fecha_fin_disponible',
       'dias_cupon_disponible', 'fecha_inicio_valido', 'fecha_fin_valido',
       'dias_cupon_valido', 'valido_lunes', 'valido_martes',
       'valido_miercoles', 'valido_jueves', 'valido_viernes', 'valido_sabado',
       'valido_domingo', 'valido_fiestas', 'valido_previo_fiestas',
       'large_area_name_cupon', 'ken_name_cupon', 'small_area_name_cupon',
       'ID_cupon_compra', 'cantidad_cupon_compra', 'fecha_compra',
       'small_area_name_compra', 'latitud_usuario', 'longitud_usuario',
       'Cap_tex_traducido_cupon', 'cupones', 'compras'],
      dtype='object')

#### One-Hot Encoding

Las variables categóricas deben ser recodificadas para poder ser utilizadas en el entrenamiento.

In [13]:
# Determinamos variables a modificar
categorical_features = ['sexo',
                       'Cap_tex_traducido_cupon']

In [14]:
# Generamos encoding
ds = pd.get_dummies(ds, prefix=None, columns=categorical_features)

In [15]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2666735 entries, 0 to 2666734
Data columns (total 69 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   compra                                         int64  
 1   fecha_visita                                   object 
 2   page_serial                                    int64  
 3   referrer_hash                                  object 
 4   ID_cupon_visto                                 object 
 5   ID_usuario                                     object 
 6   ID_sesion                                      object 
 7   ID_compra                                      object 
 8   fecha_reg_usuario                              object 
 9   edad                                           int64  
 10  fecha_baja_usuario                             object 
 11  pref_name_usuario                              object 
 12  capsule_text_cupon                        

### Nuevas variables

Previo a generar las trayectorias de cada usuario, vamos a generar algunas variables para capturar más información de los estados y que ayuden al algoritmo a identificar patrones y tendencias de los usuarios dentro del sitio.

#### Tiempo desde última visita

Tomamos visita como interaccion, por lo que pasamos a calcular la diferencia entre interacciones

In [16]:
# Transformarmos a fecha
ds['fecha_visita'] = pd.to_datetime(ds['fecha_visita'])

# Ordenamos por ID_Usuario y fecha visita de forma ascendente para luego realizar la resta entre fechas
ds.sort_values(['ID_usuario','fecha_visita'],
               axis=0,
               ascending = True,
               inplace = True,
               ignore_index = True)

In [17]:
# Calculamos la diferencia entre las visitas de los usuarios
ds['t_entre_visitas'] = ds.groupby('ID_usuario')['fecha_visita'].diff()

# Pasamos todos los valores a segundos
ds['t_entre_visitas'] = ds['t_entre_visitas'].apply(lambda x: x.total_seconds())

#### Total clics acumulados

En este caso vamos a acumular cantidad de compras realizadas y cantidad de artículos adquiridos.

In [18]:
# vemos ds con datos modificados a cero
ds[['compra', 'cantidad_cupon_compra']].fillna(0).head()

,compra,cantidad_cupon_compra
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,1,2.0


In [19]:
# Generamos modificacion
ds['cantidad_cupon_compra'] = ds['cantidad_cupon_compra'].fillna(0)

In [20]:
# Generamos la suma acumulada de compras 
ds['acumulado_compras'] = ds.groupby('ID_usuario')['compra'].cumsum()

In [21]:
# Verificamos para un IDs
ds[ds['ID_usuario'] == 'fffafc024e264d5d539813444cf61199'][['compra','acumulado_compras']].values

array([[0, 0],
       [0, 0],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 2]], dtype=int64)

In [22]:
# Generamos la suma acumulada de cupones adquiridos 
ds['acumulado_cupones'] = ds.groupby('ID_usuario')['cantidad_cupon_compra'].cumsum()

In [23]:
# Verificamos para un IDs
ds[ds['ID_usuario'] == '0083cc12fd8a104d507bc3a18c8a6013'][['cantidad_cupon_compra','acumulado_cupones']].\
                        values.tolist()

[[0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [4.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [0.0, 4.0],
 [2.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [0.0, 6.0],
 [2.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [0.0, 8.0],
 [2.0, 10.0],
 [0.0, 10.0],
 [0.0, 10.0],
 [0.0, 10.0],
 [0.0, 10.0],
 [0.0, 

#### Visitas acumuladas

La variable mira visitas acumuladas sin tener en cuenta la visita actual, por lo que su valor inicial es 0

In [24]:
# Generamos acumulado de visitas por usuario
ds['acumulado_visitas'] = ds.groupby('ID_usuario')['ID_usuario'].cumcount() # +1 para que arranque de 1

In [25]:
#verificamos
ds[ds['ID_usuario'] == 'fffafc024e264d5d539813444cf61199']['acumulado_visitas']

2666714     0
2666715     1
2666716     2
2666717     3
2666718     4
2666719     5
2666720     6
2666721     7
2666722     8
2666723     9
2666724    10
2666725    11
2666726    12
2666727    13
2666728    14
2666729    15
2666730    16
2666731    17
2666732    18
2666733    19
2666734    20
Name: acumulado_visitas, dtype: int64

#### Tiempo desde última compra

In [26]:
# Ordenamos por ID_Usuario y fecha visita de forma ascendente
ds.sort_values(['ID_usuario','fecha_visita'],
               axis=0,
               ascending = True,
               inplace = True,
               ignore_index = True)

In [27]:
# Realizamos logíca para cálculo
user = None
datos_ultima_compra = None
t_ultima_compra = []
total_rows = len(ds.index)
processed = 0
for index, row in ds.iterrows():
    processed = processed + 1
    if processed % 1000 == 0:
        print(f"{processed} procesados de {total_rows}")
    if row["ID_usuario"] != user:
        user = row["ID_usuario"]
        datos_ultima_compra = None
    if row["compra"] == 0 and datos_ultima_compra is None:
        t_ultima_compra.append(None)
    elif row["compra"] == 0 and datos_ultima_compra is not None:
        valor = row["fecha_visita"] - datos_ultima_compra
        t_ultima_compra.append(valor)
    elif row["compra"] == 1 and datos_ultima_compra is None:
        t_ultima_compra.append(0)
        datos_ultima_compra = row["fecha_visita"]
    elif row["compra"] == 1 and datos_ultima_compra is not None:
        valor = row["fecha_visita"] - datos_ultima_compra
        t_ultima_compra.append(valor)
        datos_ultima_compra = row["fecha_visita"]
#len(t_ultima_compra)
ds["t_ultima_compra"] = t_ultima_compra


1000 procesados de 2666735
2000 procesados de 2666735
3000 procesados de 2666735
4000 procesados de 2666735
5000 procesados de 2666735
6000 procesados de 2666735
7000 procesados de 2666735
8000 procesados de 2666735
9000 procesados de 2666735
10000 procesados de 2666735
11000 procesados de 2666735
12000 procesados de 2666735
13000 procesados de 2666735
14000 procesados de 2666735
15000 procesados de 2666735
16000 procesados de 2666735
17000 procesados de 2666735
18000 procesados de 2666735
19000 procesados de 2666735
20000 procesados de 2666735
21000 procesados de 2666735
22000 procesados de 2666735
23000 procesados de 2666735
24000 procesados de 2666735
25000 procesados de 2666735
26000 procesados de 2666735
27000 procesados de 2666735
28000 procesados de 2666735
29000 procesados de 2666735
30000 procesados de 2666735
31000 procesados de 2666735
32000 procesados de 2666735
33000 procesados de 2666735
34000 procesados de 2666735
35000 procesados de 2666735
36000 procesados de 2666735
3

In [28]:
print(t_ultima_compra)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
# Pasamos a segundos
ds['t_ultima_compra'] = ds['t_ultima_compra'].fillna(0).apply(lambda x: x.total_seconds() if x!=0 else x)

In [30]:
ds[ds["ID_usuario"]== "0000b53e182165208887ba65c079fc21"]

,compra,fecha_visita,page_serial,referrer_hash,ID_cupon_visto,ID_usuario,ID_sesion,ID_compra,fecha_reg_usuario,edad,...,Cap_tex_traducido_cupon_Relaxation,Cap_tex_traducido_cupon_Resort inn,Cap_tex_traducido_cupon_Spa,Cap_tex_traducido_cupon_Vacation rental,Cap_tex_traducido_cupon_Web service,t_entre_visitas,acumulado_compras,acumulado_cupones,acumulado_visitas,t_ultima_compra
0,0,2011-08-25 00:34:05,34,b9e75a3c42abddd6184aeea89a30f174,2d231a9ce88beb42b0952f59d3e0e8e8,0000b53e182165208887ba65c079fc21,2067eaaa39284c45bc8624199aea6fa8,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,NaN,0,0.0,0,0.0
1,0,2011-08-25 00:35:47,35,f4f64440290b7a391bd4c1c7ddeb7d68,d844cf6327b285f945fd93fd61b39e49,0000b53e182165208887ba65c079fc21,2067eaaa39284c45bc8624199aea6fa8,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,102.0,0,0.0,1,0.0
2,0,2011-09-23 23:43:16,14,7d3892e54acb559ae36c459978489330,38beeadfe3f97e640367eddae4a8c1b5,0000b53e182165208887ba65c079fc21,6a0574b040af9a7cb5303af95ff3591e,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,2588849.0,0,0.0,2,0.0
3,0,2011-09-23 23:46:29,17,7d3892e54acb559ae36c459978489330,38beeadfe3f97e640367eddae4a8c1b5,0000b53e182165208887ba65c079fc21,6a0574b040af9a7cb5303af95ff3591e,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,193.0,0,0.0,3,0.0
4,1,2011-09-23 23:53:45,20,6c5616ced18d79fa5a97c5c8f3e65f8f,38beeadfe3f97e640367eddae4a8c1b5,0000b53e182165208887ba65c079fc21,6a0574b040af9a7cb5303af95ff3591e,58950614c5f22543731169aa411b575b,2011-06-18 18:12:01,56,...,0,0,0,0,0,436.0,1,2.0,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,0,2012-01-27 00:09:55,12,4d5f50b7b941481f49f3025a793c1b89,3a55fd3a869b1674efa364debfa4442b,0000b53e182165208887ba65c079fc21,381e8077259da8388c230474176caeb1,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,43.0,1,2.0,70,10800970.0
71,0,2012-01-27 00:11:03,14,7d3892e54acb559ae36c459978489330,72c3429137ec83cfe48e3ab49d8c6007,0000b53e182165208887ba65c079fc21,381e8077259da8388c230474176caeb1,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,68.0,1,2.0,71,10801038.0
72,0,2012-01-27 00:12:47,15,0f92955e1abe3197f1e027082eb8c2fa,9342b8153ee0cb58248499bc55b7e8bb,0000b53e182165208887ba65c079fc21,381e8077259da8388c230474176caeb1,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,104.0,1,2.0,72,10801142.0
73,0,2012-01-27 00:18:44,16,548f101864ed16dc39a2d7438010ccd4,3c7e742af0db59fb6d714b3c6f5845b9,0000b53e182165208887ba65c079fc21,381e8077259da8388c230474176caeb1,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,0,357.0,1,2.0,73,10801499.0


#### Tiempo desde registro inicial

In [31]:
# pasamos a fecha
ds['fecha_reg_usuario'] = pd.to_datetime(ds['fecha_reg_usuario']) 

# Calculamos la diferencia
ds['t_desde_registro'] = (ds['fecha_visita'] - ds['fecha_reg_usuario'])

# Pasamos a segundos
ds['t_desde_registro'] = ds['t_desde_registro'].apply(lambda x: x.total_seconds())

In [32]:
ds[['fecha_reg_usuario', 'fecha_visita', 't_desde_registro']].head()

,fecha_reg_usuario,fecha_visita,t_desde_registro
0,2011-06-18 18:12:01,2011-08-25 00:34:05,5811724.0
1,2011-06-18 18:12:01,2011-08-25 00:35:47,5811826.0
2,2011-06-18 18:12:01,2011-09-23 23:43:16,8400675.0
3,2011-06-18 18:12:01,2011-09-23 23:46:29,8400868.0
4,2011-06-18 18:12:01,2011-09-23 23:53:45,8401304.0


In [33]:
# generamos un Dataset Interedio para luego omenzar desde aca
#ds.to_csv(link+'ds_intermedio.csv',
#          header=True,
#         index=False)

## Trayectorias finales

Para poder utilizar el DS para entrenar nuestro agente, necesitamos poder generar trayectorias. Estas trayectorias deben tener el formato (state, action, reward, done, next_state) para luego alimentar la net.

**State** = caracteristicas del usuario y cupón al momento de la compra

**Action** = Ad/cupón que se le desplegó

**Reward** = 1 si compró 0 caso contrario

**Done** = si es el último elemento de su trayectria

**Next_State** = próximo estado al usuario para casos donde Done = False

In [5]:
# Levantamos DS intermedio
ds = pd.read_csv(link+'ds_intermedio.csv',sep=',')

C:\Users\bmottini\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [34]:
ds.head()

,compra,fecha_visita,page_serial,referrer_hash,ID_cupon_visto,ID_usuario,ID_sesion,ID_compra,fecha_reg_usuario,edad,...,Cap_tex_traducido_cupon_Resort inn,Cap_tex_traducido_cupon_Spa,Cap_tex_traducido_cupon_Vacation rental,Cap_tex_traducido_cupon_Web service,t_entre_visitas,acumulado_compras,acumulado_cupones,acumulado_visitas,t_ultima_compra,t_desde_registro
0,0,2011-08-25 00:34:05,34,b9e75a3c42abddd6184aeea89a30f174,2d231a9ce88beb42b0952f59d3e0e8e8,0000b53e182165208887ba65c079fc21,2067eaaa39284c45bc8624199aea6fa8,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,NaN,0,0.0,0,0.0,5811724.0
1,0,2011-08-25 00:35:47,35,f4f64440290b7a391bd4c1c7ddeb7d68,d844cf6327b285f945fd93fd61b39e49,0000b53e182165208887ba65c079fc21,2067eaaa39284c45bc8624199aea6fa8,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,102.0,0,0.0,1,0.0,5811826.0
2,0,2011-09-23 23:43:16,14,7d3892e54acb559ae36c459978489330,38beeadfe3f97e640367eddae4a8c1b5,0000b53e182165208887ba65c079fc21,6a0574b040af9a7cb5303af95ff3591e,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,2588849.0,0,0.0,2,0.0,8400675.0
3,0,2011-09-23 23:46:29,17,7d3892e54acb559ae36c459978489330,38beeadfe3f97e640367eddae4a8c1b5,0000b53e182165208887ba65c079fc21,6a0574b040af9a7cb5303af95ff3591e,NaN,2011-06-18 18:12:01,56,...,0,0,0,0,193.0,0,0.0,3,0.0,8400868.0
4,1,2011-09-23 23:53:45,20,6c5616ced18d79fa5a97c5c8f3e65f8f,38beeadfe3f97e640367eddae4a8c1b5,0000b53e182165208887ba65c079fc21,6a0574b040af9a7cb5303af95ff3591e,58950614c5f22543731169aa411b575b,2011-06-18 18:12:01,56,...,0,0,0,0,436.0,1,2.0,4,0.0,8401304.0


In [35]:
# Veamos las variables finales para determinar cuales vamos a incluir
ds.columns

Index(['compra', 'fecha_visita', 'page_serial', 'referrer_hash',
       'ID_cupon_visto', 'ID_usuario', 'ID_sesion', 'ID_compra',
       'fecha_reg_usuario', 'edad', 'fecha_baja_usuario', 'pref_name_usuario',
       'capsule_text_cupon', 'genre_name_cupon', 'descuento_cupon',
       'precio_full', 'precio_dto', 'fecha_inicio_disponible',
       'fecha_fin_disponible', 'dias_cupon_disponible', 'fecha_inicio_valido',
       'fecha_fin_valido', 'dias_cupon_valido', 'valido_lunes',
       'valido_martes', 'valido_miercoles', 'valido_jueves', 'valido_viernes',
       'valido_sabado', 'valido_domingo', 'valido_fiestas',
       'valido_previo_fiestas', 'large_area_name_cupon', 'ken_name_cupon',
       'small_area_name_cupon', 'ID_cupon_compra', 'cantidad_cupon_compra',
       'fecha_compra', 'small_area_name_compra', 'latitud_usuario',
       'longitud_usuario', 'cupones', 'compras', 'sexo_f', 'sexo_m',
       'Cap_tex_traducido_cupon_Beauty', 'Cap_tex_traducido_cupon_Class',
       'Cap_tex_

In [36]:
# Agrupamos las variables
state_feature = [#'ID_usuario',
                 'edad',
                 'sexo_f',
                 'sexo_m',
                 'latitud_usuario',
                 'longitud_usuario',
                 'descuento_cupon',
                 'precio_full',
                 'precio_dto',
                 'dias_cupon_disponible',
                 'dias_cupon_valido',
                 'valido_lunes',
                 'valido_martes',
                 'valido_miercoles',
                 'valido_jueves',
                 'valido_viernes',
                 'valido_sabado',
                 'valido_domingo',
                 'valido_fiestas',
                 'valido_previo_fiestas',
                 'Cap_tex_traducido_cupon_Beauty',
                 'Cap_tex_traducido_cupon_Class',
                 'Cap_tex_traducido_cupon_Correspondence course',
                 'Cap_tex_traducido_cupon_Delivery service',
                 'Cap_tex_traducido_cupon_Event',
                 'Cap_tex_traducido_cupon_Food',
                 'Cap_tex_traducido_cupon_Gift card',
                 'Cap_tex_traducido_cupon_Guest house',
                 'Cap_tex_traducido_cupon_Hair salon',
                 'Cap_tex_traducido_cupon_Health and medical',
                 'Cap_tex_traducido_cupon_Hotel',
                 'Cap_tex_traducido_cupon_Japanese hotel',
                 'Cap_tex_traducido_cupon_Japanse guest house',
                 'Cap_tex_traducido_cupon_Leisure',
                 'Cap_tex_traducido_cupon_Lesson',
                 'Cap_tex_traducido_cupon_Lodge',
                 'Cap_tex_traducido_cupon_Nail and eye salon',
                 'Cap_tex_traducido_cupon_Other',
                 'Cap_tex_traducido_cupon_Public hotel',
                 'Cap_tex_traducido_cupon_Relaxation',
                 'Cap_tex_traducido_cupon_Resort inn',
                 'Cap_tex_traducido_cupon_Spa',
                 'Cap_tex_traducido_cupon_Vacation rental',
                 'Cap_tex_traducido_cupon_Web service',
                 't_entre_visitas',
                 'acumulado_compras',
                 'acumulado_cupones',
                 'acumulado_visitas',
                 't_ultima_compra',
                 't_desde_registro']

In [37]:
action = ['ID_cupon_visto']

In [38]:
reward = ['compra']

### Tipos y Redondeos

Verificamos tipos de variables y valores para poder posteriormente poder utilizarlos

In [39]:
# realzamos un fillna() con ceros, esto es un punto a RECONSIDERRAR
ds = ds.fillna(0)

In [40]:
# Pasamos todo a Int
ds[state_feature] = ds[state_feature].astype(int)
ds[reward] = ds[reward].astype(int)

In [41]:
ds[state_feature].head()

,edad,sexo_f,sexo_m,latitud_usuario,longitud_usuario,descuento_cupon,precio_full,precio_dto,dias_cupon_disponible,dias_cupon_valido,...,Cap_tex_traducido_cupon_Resort inn,Cap_tex_traducido_cupon_Spa,Cap_tex_traducido_cupon_Vacation rental,Cap_tex_traducido_cupon_Web service,t_entre_visitas,acumulado_compras,acumulado_cupones,acumulado_visitas,t_ultima_compra,t_desde_registro
0,56,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5811724
1,56,1,0,0,0,63,11550,4200,3,179,...,0,0,0,0,102,0,0,1,0,5811826
2,56,1,0,0,0,62,7900,2980,3,179,...,0,0,0,0,2588849,0,0,2,0,8400675
3,56,1,0,0,0,62,7900,2980,3,179,...,0,0,0,0,193,0,0,3,0,8400868
4,56,1,0,0,0,62,7900,2980,3,179,...,0,0,0,0,436,1,2,4,0,8401304


### Funcion 1

Esta primera funcion genera una lista que contiene para para cada interacción del usuario una lista con [estado, acccion, reward, done, proximo estado]

In [42]:
def genera_tupla(interaccion):
    tupla = []
    
    tupla.append(ds[state_feature].loc[interaccion].values.tolist())
    tupla.append(ds[action].loc[interaccion].values.tolist())
    tupla.append(ds[reward].loc[interaccion].values.tolist())
    try:
        if len(ds[state_feature].loc[interaccion+1].values.tolist()) > 0:
            tupla.append(0) # La red necesita 1 y 0 y no True o False
            tupla.append(ds[state_feature].loc[interaccion+1].values.tolist())
        
        
    except:
        tupla.append(1) # La red necesita 1 y 0 y no True o False
        tupla.append('')
        
    
    return tupla

In [43]:
def genera_trayectoria_usuario(usuario):
    trayectoria = []
    
    for x in list(ds[ds['ID_usuario'] == usuario].index):
        trayectoria.append(genera_tupla(x))
    
    #print('Fin procesamiento trayectoria usuario:', usuario)
    return trayectoria

In [16]:
prueba1 = genera_trayectoria_usuario('fffafc024e264d5d539813444cf61199')

In [17]:
print(prueba1)

[[[66, 0, 1, 34, 138, 100, 1, 0, 4, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -189], ['09411858ae07c0be91aeeddacf4556b4'], [0], 0, [66, 0, 1, 34, 138, 100, 1, 0, 4, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 420, 0, 0, 1, 0, 231]], [[66, 0, 1, 34, 138, 100, 1, 0, 4, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 420, 0, 0, 1, 0, 231], ['09411858ae07c0be91aeeddacf4556b4'], [0], 0, [66, 0, 1, 34, 138, 100, 1, 0, 4, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 211, 1, 1, 2, 0, 442]], [[66, 0, 1, 34, 138, 100, 1, 0, 4, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 211, 1, 1, 2, 0, 442], ['09411858ae07c0be91aeeddacf4556b4'], [1], 0, [66, 0, 1, 34, 138, 54, 2220, 1000, 3, 0, 0, 0, 0, 0, 0, 

In [18]:
prueba1[0][1]

['09411858ae07c0be91aeeddacf4556b4']

### Funcion 2

El enfoque de esta funcion es diferente, en este caso generamos los elementos de la trayectoria por separado en forma batch, por lo que el resultado de la funcion son 5 elementos (listas), donde cada una contiene el total de la informacion de dicho elemento.

Ej: reward = [0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1]

In [44]:
# Funcion que crea tuplas con la información
def elementos_trayectoria(usuario):
    state_list = []
    action_list = []
    reward_list = []
    done_list = []
    next_state_list = []
    
    for x in list(ds[ds['ID_usuario'] == usuario].index):
        state_list.append(ds[state_feature].loc[x].values.tolist())
        action_list.append(ds[action].loc[x].values.tolist())
        reward_list.append(ds[reward].loc[x].values.tolist())
        try:
            next_state_list.append(ds[state_feature].loc[x+1].values.tolist())
            done_list.append((0)) # La red necesita 1 y 0 y no True o False
        except:
            next_state_list.append('')
            done_list.append((1)) # La red necesita 1 y 0 y no True o False
    
    print('Fin procesamiento trayectoria usuario:', usuario)
    return state_list, action_list, reward_list, done_list, next_state_list    

In [20]:
state_b, action_b, reward_b, done_b, next_state_list = elementos_trayectoria('fffafc024e264d5d539813444cf61199')

Fin procesamiento trayectoria usuario: fffafc024e264d5d539813444cf61199


In [21]:
reward_b, done_b

([[0],
  [0],
  [1],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [1]],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

###### Ambas funciones previamente planteadas son muy lentas a la hora de procesar la información, demorando de forma aproximada 1 segundo por tupla generada, lo que llevaria 2.5M de segundos (700hs) generar el total del DS. 

Intentaremos generar otra solución para generar las tuplas y DS final con el formato necesario

### Funcion 3

La ides es ir creando listas con los datos necesarios y luego iterar sobre estas listas para que el proceso sea más rápido

In [45]:
# Ordenamos por ID_Usuario y fecha visita de forma ascendente
ds.sort_values(['ID_usuario','fecha_visita'],
               axis=0,
               ascending = True,
               inplace = True,
               ignore_index = True)

In [46]:
def process_state(obs):
    return torch.from_numpy(obs).float()

In [47]:
def genera_v_done(serie):
    lista_done = []
    total = 0
    for x in range(len(serie)):
        try:
            if serie[x] == serie[x+1]:
                lista_done.append(0)
            else:
                lista_done.append(1)
        except:
            lista_done.append(1)
        
        total += 1
        
        if total % 1000 == 0:
            print(total)
    return lista_done

In [22]:
# Aplicamos la función y sumamos a ds
usuarios = ds.ID_usuario.values.tolist()
variable_done = genera_v_done(usuarios)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [23]:
# Agregamos nueva variable al DS para ver si está correcto
ds['done'] = variable_done
ds[ds['ID_usuario'] == 'fffafc024e264d5d539813444cf61199'][['ID_usuario','done']].tail()

,ID_usuario,done
2666730,fffafc024e264d5d539813444cf61199,0
2666731,fffafc024e264d5d539813444cf61199,0
2666732,fffafc024e264d5d539813444cf61199,0
2666733,fffafc024e264d5d539813444cf61199,0
2666734,fffafc024e264d5d539813444cf61199,1


In [24]:
# Ahora generamos nueva funcion para la variable Next_State
def genera_v_next_state(listado_estados, listado_done):
    
    next_states = []
    total = 0
    
    for x in range(len(listado_estados)):
        
        if listado_done[x] == 1:
            next_states.append('')
        else:
            next_states.append(process_state(np.array(listado_estados[x+1])))
        total += 1
        if total % 1000 == 0:
            print(total)
   
    return next_states

In [25]:
# Definimos las listas sobre las cuales iterar
variable_done # Previamente creada
estados = ds[state_feature].values.tolist()

In [26]:
# Generamos variable
variable_ns = genera_v_next_state(estados,variable_done)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [27]:
# veamos un ejemplo para chequear este OK y comparamos con ds
ds[state_feature].tail()

,edad,sexo_f,sexo_m,latitud_usuario,longitud_usuario,descuento_cupon,precio_full,precio_dto,dias_cupon_disponible,dias_cupon_valido,...,Cap_tex_traducido_cupon_Resort inn,Cap_tex_traducido_cupon_Spa,Cap_tex_traducido_cupon_Vacation rental,Cap_tex_traducido_cupon_Web service,t_entre_visitas,acumulado_compras,acumulado_cupones,acumulado_visitas,t_ultima_compra,t_desde_registro
2666730,66,0,1,34,138,52,4200,2000,3,0,...,0,0,0,0,63,1,1,16,192057,192499
2666731,66,0,1,34,138,56,3600,1580,3,0,...,0,0,0,0,427745,1,1,17,619802,620244
2666732,66,0,1,34,138,75,8050,1995,4,0,...,0,0,0,0,292,1,1,18,620094,620536
2666733,66,0,1,34,138,53,3448,1600,3,0,...,0,0,0,0,142,1,1,19,620236,620678
2666734,66,0,1,34,138,53,3448,1600,3,0,...,0,0,0,0,182,2,2,20,620418,620860


In [28]:
print(variable_ns[-5:])

[tensor([6.6000e+01, 0.0000e+00, 1.0000e+00, 3.4000e+01, 1.3800e+02, 5.6000e+01,
        3.6000e+03, 1.5800e+03, 3.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 4.2774e+05, 1.0000e+00, 1.0000e+00, 1.7000e+01, 6.1980e+05,
        6.2024e+05]), tensor([6.6000e+01, 0.0000e+00, 1.0000e+00, 3.4000e+01, 1.3800e+02, 7.5000e+01,
        8.0500e+03, 1.9950e+03, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+0

In [29]:
# Generamos variables faltantes
acciones = ds.ID_cupon_visto.values.tolist()
rewards = ds.compra.values.tolist()

In [30]:
#Paso a tensores estados iniciales
for x in range(len(estados)):
    estados[x] = process_state(np.array(estados[x]))

In [31]:
# Generamos treyectorias finales
trayectorias_fianles = []
for estado, accion, reward, done, next_state in zip(estados,acciones,rewards,variable_done,variable_ns):
    trayectorias_fianles.append((estado,accion,reward,done,next_state))

In [32]:
len(trayectorias_fianles)

2666735

In [33]:
trayectorias_fianles[15489][1]

'212c05b62741005bd904a8e267972c18'

In [34]:
# generamos archivo en formato pickle
with open("C:/Users/bmottini/Desktop/Master 2020/Tesis/Dataset/trayectorias_finales.txt", "wb") as fp:
    pickle.dump(trayectorias_fianles, fp)

In [35]:
# Probamos levantarlo
with open("C:/Users/bmottini/Desktop/Master 2020/Tesis/Dataset/trayectorias_finales.txt", "rb") as fp:
    trayectorias_finales_carga = pickle.load(fp)

In [36]:
len(trayectorias_finales_carga)

2666735

In [37]:
trayectorias_finales_carga[15489][1]

'212c05b62741005bd904a8e267972c18'